In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
# Load the dataset
data = pd.read_csv("keypoint.csv", header=None)

In [3]:
# Ensure the labels (first column) are strings
data[0] = data[0].astype(str)

In [4]:
# Check unique labels to verify data integrity
print(data[0].unique())

['bathroom' 'hello' 'help' 'iloveyou' 'more' 'no' 'repeat' 'thanks' 'yes']


In [5]:
# Features (X) and labels (y)
X = data.iloc[:, 1:]
y = data[0]

In [6]:
# Encode the labels
enc = LabelEncoder()
y = enc.fit_transform(y)

In [7]:
# Print the encoded labels to verify
print(y)

[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3
 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8]


In [8]:
# Callback for early stopping
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Define the model
model = keras.Sequential([
    layers.Dense(1470, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(832, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(428, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(264, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(enc.classes_), activation='softmax')
])

In [11]:
# Compile the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=128, validation_split=0.2, callbacks=[es])

Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 2.2256 - accuracy: 0.1286 - val_loss: 2.1610 - val_accuracy: 0.1111
Epoch 2/50
1/1 [==============================] - 0s 45ms/step - loss: 2.1364 - accuracy: 0.1857 - val_loss: 2.1408 - val_accuracy: 0.1111
Epoch 3/50
1/1 [==============================] - 0s 50ms/step - loss: 2.1351 - accuracy: 0.1857 - val_loss: 2.1411 - val_accuracy: 0.1111
Epoch 4/50
1/1 [==============================] - 0s 45ms/step - loss: 2.0718 - accuracy: 0.2143 - val_loss: 2.1367 - val_accuracy: 0.1111
Epoch 5/50
1/1 [==============================] - 0s 49ms/step - loss: 1.9979 - accuracy: 0.2286 - val_loss: 2.1323 - val_accuracy: 0.1111
Epoch 6/50
1/1 [==============================] - 0s 54ms/step - loss: 1.9989 - accuracy: 0.2143 - val_loss: 2.1228 - val_accuracy: 0.1111
Epoch 7/50
1/1 [==============================] - 0s 50ms/step - loss: 2.0279 - accuracy: 0.2286 - val_loss: 2.1039 - val_accuracy: 0.1111
Epoch 8/50
1/1 [=============

In [13]:
# Evaluate the model
model.evaluate(X_test, y_test, verbose=0)

[0.2534199357032776, 0.9130434989929199]

In [14]:
# Predictions on the test set
y_val_pred = model.predict(X_test)
y_val_pred_classes = np.argmax(y_val_pred, axis=1)

1/1 [==============================] - 0s 96ms/step


In [15]:
# Performance metrics
acc = accuracy_score(y_test, y_val_pred_classes)
prec = precision_score(y_test, y_val_pred_classes, average='macro')
rec = recall_score(y_test, y_val_pred_classes, average='macro')
f1 = f1_score(y_test, y_val_pred_classes, average='macro')

In [16]:
# Print performance metrics
print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1-score:", f1)

Accuracy: 0.9130434782608695
Precision: 0.9166666666666666
Recall: 0.9682539682539684
F1-score: 0.9285714285714286


In [17]:
# Save the model
model.save("model.h5")